In [ ]:
pip install streamlit pymupdf openai pytesseract pillow faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

In [ ]:
%%writefile app_final.py

import streamlit as st
import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from PIL import Image
import io
import openai
import pytesseract
import os

# Load the embedding model
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to extract text and images from a PDF
def extract_text_and_images_from_pdf(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    page_data = []

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")
        pix = doc[page_num].get_pixmap()  # Extract page image as a pixmap
        img = Image.open(io.BytesIO(pix.tobytes("png")))  # Convert to PIL Image
        page_data.append({"text": text, "image": img})

    return page_data

# Function to create FAISS index
def create_faiss_index(page_data):
    text_data = [page["text"] for page in page_data]
    embeddings = np.array([embed_model.encode(text) for text in text_data]).astype('float32')

    # Create FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, page_data

# Function to search for the most relevant page
def search_pdf(question, index, page_data):
    question_embedding = np.array([embed_model.encode(question)]).astype('float32')
    D, I = index.search(question_embedding, k=1)  # Get the closest match

    if I[0][0] != -1:
        page_num = I[0][0]
        return page_num, page_data[page_num]["text"], page_data[page_num]["image"]
    return None, "No relevant information found.", None

#Function to perform OCR on an image
def ocr_image(image):
    text = pytesseract.image_to_string(image)
    return text

# Function to get the GPT-4 answer
def get_gpt4_answer(question, context):
    prompt = f"Answer the following question based on the given context:\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",  # Ensure you're using GPT-4
        messages=messages,
        max_tokens=200,
        temperature=0.7
    )

    return response['choices'][0]['message']['content'].strip()

# Streamlit UI
st.title("Math PDF Q&A System")

uploaded_pdf = st.file_uploader("Upload a Math PDF", type="pdf")

if uploaded_pdf:
    st.success("PDF uploaded successfully! Extracting text and images...")
    page_data = extract_text_and_images_from_pdf(uploaded_pdf)
    index, page_data = create_faiss_index(page_data)
    st.success("Indexing complete. You can now ask questions.")

    #question = st.text_input("Ask a question from the PDF:")

    # Asking a question
    question_type = st.radio("Select question type", ("Text", "Image"))

    if question_type == "Text":
        question = st.text_input("Ask a question:")
        page_num, answer, page_image = search_pdf(question, index, page_data)
        if page_num is not None:
            st.subheader(f"Answer (Page {page_num + 1}):")
            st.write(answer)
            st.image(page_image, caption=f"Page {page_num + 1}")
        else:
            st.error("No relevant information found.")
    elif question_type == "Image":
        uploaded_image = st.file_uploader("Upload your question image")

        if uploaded_image:
            image = Image.open(uploaded_image)
            question = ocr_image(image)
            st.image(image, caption="Uploaded Image")
            st.write(f"Extracted Question: {question}")
            page_num, answer, page_image = search_pdf(question, index, page_data)
            if page_num is not None:
               st.subheader(f"Answer (Page {page_num + 1}):")
               st.write(answer)
               st.image(page_image, caption=f"Page {page_num + 1}")
            else:
               st.error("No relevant information found.")




Overwriting app_final.py


In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.66.3
    Uninstalling openai-1.66.3:
      Successfully uninstalled openai-1.66.3


In [ ]:
# Install Tesseract
!apt-get install tesseract-ocr

# Install pytesseract
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
! pip install streamlit -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.23.76.201


In [29]:
!streamlit run app_final.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.76.201:8501

⠋⠙⠹your url is: https://warm-windows-lick.loca.lt
2025-03-21 09:24:26.077308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742549066.108655   35404 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742549066.117957   35404 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 09:24:26.155002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the fo